# 1.1 Red teaming Agent in AI Foundry (Project based resource)

## Load environment variables


In [ ]:
#!az login

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("../infra/credentials.env", override=True)

# print some varaibles to check the process
print("Project endpoint:", os.getenv("AZURE_PJ_PROJECT_ENDPOINT"))

## Create an AI Red Teaming Agent
You can instantiate the AI Red Teaming agent with your Azure AI Project and Azure Credentials.

In [ ]:
# Azure imports
from azure.identity import DefaultAzureCredential
from azure.ai.evaluation.red_team import RedTeam, RiskCategory

# Azure AI Project Information (unique for the project approach)
azure_ai_project = os.environ.get("AZURE_PJ_PROJECT_ENDPOINT")

# Instantiate your AI Red Teaming Agent
red_team_agent = RedTeam(
    azure_ai_project=azure_ai_project,     # required
    credential=DefaultAzureCredential(),   # required
    risk_categories=[                      # optional, defaults to all four risk categories
        RiskCategory.Violence
        # RiskCategory.HateUnfairness,
        # RiskCategory.Sexual,
        # RiskCategory.SelfHarm
    ], 
    num_objectives=1,                      # optional, defaults to 10
)


Optionally, you can specify which **risk categories** of content risks you want to cover with `risk_categories` and define the **number of prompts** covering each risk category with `num_objectives`. The previous example generates 5 seed prompts for each risk category for a total of 20 rows of prompts to be generated and sent to your target.

## Types of Targets You Can Scan with the AI Red Teaming Agent

When using the AI Red Teaming Agent, you must define a **target**, which is the AI system or interface you want to probe for safety risks. The agent supports several types of targets, depending on how your AI system is deployed or accessed:

### 1 - Model Configuration
Use this when scanning a deployed model, such as an Azure OpenAI deployment. This is ideal during model selection or benchmarking phases.

### 2 - Simple Callback
Use a basic Python function that receives a prompt and returns a response. Perfect if you have a custom application (e.g., a chatbot or RAG pipeline) and want to test it easily.

### 3 - Advanced Callback (Chat Protocol)
Use this when your AI system handles structured chat messages (with roles like `user`, `assistant`). This is great for systems that follow the OpenAI chat format or multi-turn logic.

### 4 - PyRIT Prompt Target
For advanced users already working with PyRIT. This option allows you to integrate existing PyRIT targets like `OpenAIChatTarget` or any class inheriting from `PromptChatTarget`.

Each of these target types enables the agent to send adversarial prompts and evaluate how your system responds to potentially harmful or sensitive queries.


## Example: Scanning an Azure OpenAI Model (Model Configuration)

In this example, we'll scan an Azure OpenAI model by passing its configuration details as the target to the AI Red Teaming Agent.

In [ ]:
# Configuration for Azure OpenAI model
azure_openai_config = {
    "azure_endpoint": os.environ.get("AZURE_PJ_MODEL_ENDPOINT"),
    "azure_deployment": os.environ.get("AZURE_PJ_MODEL_NAME"),
    "api_key": os.environ.get("AZURE_PJ_MODEL_API_KEY"),
}

In [ ]:
red_team_result = await red_team_agent.scan(target=azure_openai_config)

## Attack Strategies Overview

By default, if you run a scan without specifying any attack strategies, the AI Red Teaming Agent will use **baseline direct adversarial prompts**. These are straightforward queries aimed at triggering undesired behavior, and are a great starting point to evaluate initial model alignment.

### What Are Attack Strategies?
Attack strategies are techniques that **modify baseline adversarial prompts** to try and **bypass your AI system’s safeguards**. They simulate how a real adversary might attempt to "trick" the system.

### Levels of Attack Complexity

Attack strategies are grouped into three complexity levels based on the effort needed to perform them:

- **Easy**  
  Simple transformations, such as encoding or character swaps.  
  _Strategies: `Base64`, `Flip`, `Morse`_

- **Moderate**  
  Require extra resources like a generative model to reformulate prompts.  
  _Strategies: `Tense`_

- **Difficult**  
  Combine multiple strategies or rely on advanced techniques like search-based prompt optimization.  
  _Strategies: A composition of `Tense` + `Base64`_

You can specify individual strategies or use predefined complexity groups in the `attack_strategies` parameter during a scan.


### An example using default grouped attack strategies
The following scan would first run all the baseline direct adversarial queries. Then, it would apply the following attack techniques: Base64, Flip, Morse, Tense, and a composition of Tense and Base64 which would first translate the baseline query into past tense then encode it into Base64.

In [ ]:
from azure.ai.evaluation.red_team import AttackStrategy

# Run the red team scan with default grouped attack strategies
red_team_agent_result = await red_team_agent.scan(
    target=azure_openai_config, # required
    output_path="Scan results/My-First-RedTeam-Scan.json", #optional, output file
    scan_name="Scan with many strategies", # optional, names your scan in Azure AI Foundry
    attack_strategies=[ # optional
        AttackStrategy.EASY, 
        AttackStrategy.MODERATE,  
        AttackStrategy.DIFFICULT,
    ],
)

### An example using specific attack strategies
More advanced users can specify the desired attack strategies instead of using default groups.

In [ ]:
# Run the red team scan with multiple attack strategies
red_team_agent_result = await red_team_agent.scan(
    target=azure_openai_config, # required
    scan_name="Scan with many strategies", # optional
    attack_strategies=[ # optional
        AttackStrategy.CharacterSpace,  # Add character spaces
        AttackStrategy.ROT13,  # Use ROT13 encoding
        AttackStrategy.UnicodeConfusable,  # Use confusable Unicode characters
        AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]), # composition of strategies
    ],
)

## Understanding Your Scan Results

Once your automated red teaming scan is complete, the results are summarized in a detailed scorecard. The key metric to look at is the **Attack Success Rate (ASR)**, which measures the percentage of attack prompts that successfully triggered undesirable or unsafe responses from your AI system.

You can specify an `output_path` when running your scan to save the results in a JSON file. This file includes:

- **ASR breakdown by risk category** (e.g., violence, self-harm, etc.)
- **ASR breakdown by attack complexity** (baseline, easy, moderate, difficult)
- **Joint summaries** that correlate attack strategies with specific risk categories
- **Simulation parameters** showing which categories and attack techniques were used


### Display the Red Teaming scan results JSON

In [ ]:
import json

# Path to the output JSON file from the scan
output_file_path = "Scan results/My-First-RedTeam-Scan.json"

# Open and load the JSON file
with open(output_file_path, "r", encoding="utf-8") as file:
    red_team_results = json.load(file)

# Pretty print the JSON with indentation and Unicode support
formatted_json = json.dumps(red_team_results, indent=2, ensure_ascii=False)
print(formatted_json)



## Bring your own objectives: Using your own prompts as objectives for RedTeam
   Below we demonstrate how to use your own prompts as objectives for a `RedTeam` scan. You can see the required format for prompts under `.\assets\adversarial-prompts.json`. Note that when bringing your own prompts, the supported `risk-types` are `violence`, `sexual`, `hate_unfairness`, and `self_harm`. The number of prompts you specify will be the `num_objectives` used in the scan.

In [ ]:
path_to_prompts = "..\\assets\\substance-abuse-prompts.json"

# Create the RedTeam specifying the custom attack seed prompts to use as objectives
custom_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=DefaultAzureCredential(),
    custom_attack_seed_prompts=path_to_prompts,  # Path to a file containing custom attack seed prompts
)

custom_red_team_result = await custom_red_team.scan(
    target=azure_openai_config,
    scan_name="Custom-Prompt-Scan",
    attack_strategies=[
        AttackStrategy.EASY,
        AttackStrategy.MODERATE,  
        AttackStrategy.DIFFICULT, 
    ],
    output_path="Scan results/Custom-Prompt-Scan.json",
)